### Meta-trajectories
***

**Jupyter Notebook** designed to demonstrate the power of the **hexABC REST-API programmatic interface**. 

The **REST API** is used to extract **trajectory fragments** and construct **meta-trajectories**. These **meta-trajectories** can then be analyzed to generate **meta-analyses** that are not directly available in the database.

The workflow is powered by the [hexABC database REST API](https://mmb.irbbarcelona.org/webdev3/hexABC/rest)
***


### Importing auxiliary libraries

In [40]:
import requests
import urllib
import json
import plotly
import itertools
import ipywidgets
import nglview
from IPython.display import display
from math import ceil

## Initializing colab
The two cells below are used only in case this notebook is executed via **Google Colab**. Take into account that, for running conda on **Google Colab**, the **condacolab** library must be installed. As [explained here](https://pypi.org/project/condacolab/), the installation requires a **kernel restart**, so when running this notebook in **Google Colab**, don't run all cells until this **installation** is properly **finished** and the **kernel** has **restarted**.

In [ ]:
# Only executed when using google colab
import sys
if 'google.colab' in sys.modules:
  import subprocess
  from pathlib import Path
  try:
    subprocess.run(["conda", "-V"], check=True)
  except FileNotFoundError:
    subprocess.run([sys.executable, "-m", "pip", "install", "condacolab"], check=True)
    import condacolab
    condacolab.install()
    # Clone repository
    repo_URL = "https://github.com/mmb-irb/hexabc-meta-analyses.git"
    repo_name = Path(repo_URL).name.split('.')[0]
    if not Path(repo_name).exists():
      subprocess.run(["mamba", "install", "-y", "git"], check=True)
      subprocess.run(["git", "clone", repo_URL], check=True)
      print("⏬ Repository properly cloned.")
    # Install environment
    print("⏳ Creating environment...")
    env_file_path = f"{repo_name}/conda_env/environment.yml"
    subprocess.run(["mamba", "env", "update", "-n", "base", "-f", env_file_path], check=True)
    print("🎨 Install NGLView dependencies...")
    subprocess.run(["mamba", "install", "-y", "-c", "conda-forge", "nglview==3.0.8", "ipywidgets=7.7.2"], check=True)
    print("👍 Conda environment successfully created and updated.")

In [ ]:
# Enable widgets for colab
if 'google.colab' in sys.modules:
  from google.colab import output
  output.enable_custom_widget_manager()

### Defining auxiliary functions

In [41]:
#
# find_seq: finding MD simulations containing a sequence fragment; Returns metadata for the systems found.
#
def find_seq(json_data, pattern):
    complement_map = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    complement = ''.join(complement_map[base] for base in reversed(pattern))

    matching_projects = []

    for project in json_data:
        watson_seq = project.get('sequences', [None])[0]

        if watson_seq and len(watson_seq) > 4:
            trimmed_seq = watson_seq[2:-2]  # Skip first and last 2 bases (flanking regions)
            positions = []

            #for motif in [pattern, complement]:
            for motif in [pattern]:
                pos = trimmed_seq.find(motif)
                while pos != -1:
                    # Adjust position relative to original sequence
                    positions.append((motif, pos + 2))
                    pos = trimmed_seq.find(motif, pos + 1)

            if positions:
                matching_projects.append({
                    'id': project['id'],
                    'name': project['name'],
                    'sequence': watson_seq,
                    'positions': positions
                })

    return matching_projects

### Base REST-API URL

In [42]:
API_BASE_URL = "https://mmb.irbbarcelona.org/webdev3/hexABC/api"

### Getting projects info

Retrieving all the **projects metadata** from the **hexABC database**.

* Endpoint used: https://mmb.irbbarcelona.org/webdev3/hexABC/api/projects

In [43]:
url_get_projects = f'{API_BASE_URL}/projects?limit=1000'
with urllib.request.urlopen(url_get_projects) as response:
    r_projects = json.loads(response.read().decode("utf-8"))
print(json.dumps(r_projects, indent=4))

{
    "total": 380,
    "projects": [
        {
            "id": "seq001-1",
            "accession": "seq001-1",
            "name": "hexABC Sequence 001 - replica 1",
            "sequences": [
                "GCAAACTTGTATATGTGTGC",
                "GCACACATATACAAGTTTGC"
            ],
            "analyses": [
                "clusters",
                "clusters-00",
                "clusters-01",
                "dist-perres",
                "energies",
                "fluctuation",
                "hbonds",
                "interactions",
                "pca",
                "rgyr",
                "rmsd-pairwise",
                "rmsd-perres",
                "rmsds",
                "sasa",
                "helical"
            ]
        },
        {
            "id": "seq001-2",
            "accession": "seq001-2",
            "name": "hexABC Sequence 001 - replica 2",
            "sequences": [
                "GCAAACTTGTATATGTGTGC",
                "GCACACATATACAAGTTT

### Pagination

If the number of **projects** exceeds the default limit (50), **pagination** is needed. In this case, we need to loop over the returned pages to store all the desired information.  


In [44]:
# Set a list to store all the mined metadata
projects_metadata = []

# Set a list to store all the mined accession values
accessions = []

# Get the number of projects from the previous response
n_projects = r_projects['total']

# Set the limit of projects per page
limit = 100

# Calculate the expected number of pages
pages = ceil(n_projects / limit)

# Iterate over pages
for page in range(1, pages + 1):
    
    print(f'Requesting page {page}/{pages}', end='\r')
    
    # Set the URL for the projects endpoint
    # Include both limit and page parameters
    paginated_url = f'{API_BASE_URL}/projects?limit={limit}&page={page}'
    
    # Query the API
    with urllib.request.urlopen(paginated_url) as resp:
        response = json.loads(resp.read().decode("utf-8"))
        
        # Mine target data
        projects = response['projects']
        project_accessions = [ project['accession'] for project in projects]
        accessions += project_accessions
        projects_metadata = [*projects_metadata, *projects]
    
print(f'We have obtained metadata information for {len(accessions)} simulations')

We have obtained metadata information for 380 simulations


## TRIMERS

### Select the trimers

Selecting the desired **trimers** from the list of all possible **DNA trimers** (e.g. AAA)

In [47]:
# Generate all possible DNA trimers
bases = ['A', 'T', 'C', 'G']
trimer_list = [''.join(p) for p in itertools.product(bases, repeat=3)]

mdsel = ipywidgets.Dropdown(
    options=trimer_list,
    description='Sel. trimer:',
    disabled=False,
    value='AAA' # default value
)
display(mdsel)

Dropdown(description='Sel. trimer:', options=('AAA', 'AAT', 'AAC', 'AAG', 'ATA', 'ATT', 'ATC', 'ATG', 'ACA', '…

### Searching for trimers

Looking for specific **trimeric sequence** within the **dataset**. 


In [67]:
trimer = mdsel.value
results = find_seq(projects_metadata, trimer)

for result in results:
    print(f"{result['id']} - {result['name']}")
    print(f"  Watson strand: {result['sequence']}")
    for motif, pos in result['positions']:
        print(f"    ↳ found '{motif}' at position {pos}")


seq004-1 - hexABC Sequence 004 - replica 1
  Watson strand: GCAAATTTGGAGTCTAGAGC
    ↳ found 'AAT' at position 3
seq004-2 - hexABC Sequence 004 - replica 2
  Watson strand: GCAAATTTGGAGTCTAGAGC
    ↳ found 'AAT' at position 3
seq010-1 - hexABC Sequence 010 - replica 1
  Watson strand: GCAAGTGGGAAGAGAAATGC
    ↳ found 'AAT' at position 15
seq010-2 - hexABC Sequence 010 - replica 2
  Watson strand: GCAAGTGGGAAGAGAAATGC
    ↳ found 'AAT' at position 15
seq011-1 - hexABC Sequence 011 - replica 1
  Watson strand: GCAATGATAGCCTGTAATGC
    ↳ found 'AAT' at position 2
    ↳ found 'AAT' at position 15
seq011-2 - hexABC Sequence 011 - replica 2
  Watson strand: GCAATGATAGCCTGTAATGC
    ↳ found 'AAT' at position 2
    ↳ found 'AAT' at position 15
seq012-1 - hexABC Sequence 012 - replica 1
  Watson strand: GCAATGTCCAGGGGCGGAGC
    ↳ found 'AAT' at position 2
seq012-2 - hexABC Sequence 012 - replica 2
  Watson strand: GCAATGTCCAGGGGCGGAGC
    ↳ found 'AAT' at position 2
seq015-1 - hexABC Sequence 0

### Extract trajectories fragments

For each **trimer** found, extract the **trajectory fragment** for the specific central **base pair** of the **trimer** (e.g. A**A**T). <br> Building a **meta-trajectory** from all the fragments.  

Endpoints used:
* https://mmb.irbbarcelona.org/webdev3/hexABC/api/projects/{id}/structure
* https://mmb.irbbarcelona.org/webdev3/hexABC/api/projects/{id}/trajectory

In [68]:
import os
from glob import glob

# Clean existing sub-trajectories
old_traj_files = sorted(glob("*_backbone.xtc"))
for f in old_traj_files:
    os.remove(f)
        
# Base topology
entry = results[0]
pos = entry['positions'][0][1] 
#ini = pos + 3 # pentamer case
ini = pos + 2 # trimer case
#ini = pos + 1 # monomer case
end = 40 - ini + 1

url_param = f"{API_BASE_URL}/projects/{entry['id']}/structure?selection={ini}%20or%20{end}"

print(url_param)

with urllib.request.urlopen(url_param) as response:
    struct = response.read()

# Filename 
topology = f"{entry['sequence']}_backbone.pdb"

# Write the binary content to disk
with open(topology, 'wb') as f:
    f.write(struct)

print(f"Structure saved as {topology}")

for result in results:
    print(f"{result['id']} - {result['name']}")
    print(f"  Watson strand: {result['sequence']}")
    
    for motif, pos in result['positions']:
        print(f"    ↳ found '{motif}' at position {pos}")

        #ini = pos + 3 # pentamer case
        ini = pos + 2 # trimer case
        #ini = pos + 1 # monomer case
        end = 40 - ini + 1
        url_param = f"{API_BASE_URL}/projects/{result['id']}/trajectory?format=xtc&frames=1:450000:500&selection={ini}%20or%20{end}"
        
        print(url_param)
        
        with urllib.request.urlopen(url_param) as response:
            traj = response.read()
    
        # Filename 
        filename = f"{result['id']}_{pos}_backbone.xtc"
        
        # Write the binary content to disk
        with open(filename, 'wb') as f:
            f.write(traj)
        
        print(f"Trajectory saved as {filename}")

https://mmb.irbbarcelona.org/webdev3/hexABC/api/projects/seq004-1/structure?selection=5%20or%2036
Structure saved as GCAAATTTGGAGTCTAGAGC_backbone.pdb
seq004-1 - hexABC Sequence 004 - replica 1
  Watson strand: GCAAATTTGGAGTCTAGAGC
    ↳ found 'AAT' at position 3
https://mmb.irbbarcelona.org/webdev3/hexABC/api/projects/seq004-1/trajectory?format=xtc&frames=1:450000:500&selection=5%20or%2036
Trajectory saved as seq004-1_3_backbone.xtc
seq004-2 - hexABC Sequence 004 - replica 2
  Watson strand: GCAAATTTGGAGTCTAGAGC
    ↳ found 'AAT' at position 3
https://mmb.irbbarcelona.org/webdev3/hexABC/api/projects/seq004-2/trajectory?format=xtc&frames=1:450000:500&selection=5%20or%2036
Trajectory saved as seq004-2_3_backbone.xtc
seq010-1 - hexABC Sequence 010 - replica 1
  Watson strand: GCAAGTGGGAAGAGAAATGC
    ↳ found 'AAT' at position 15
https://mmb.irbbarcelona.org/webdev3/hexABC/api/projects/seq010-1/trajectory?format=xtc&frames=1:450000:500&selection=17%20or%2024
Trajectory saved as seq010-1_1

### Building a meta-trajectory

Taking all the **trajectory fragments**, putting them all together and aligning all frames to the first one. The resulting file is the meta-trajectory with all conformations of the specific central base-pair of the specific trimer found in the whole hexABC dataset.  

Combining all **trajectory fragments** into a **single file** and **aligning** every frame to the first one yields a **meta-trajectory**. This **meta-trajectory** captures all **conformations** of the specific **central base pair** within the specified **trimer**, as observed across the entire **hexABC dataset**.

In [69]:
import MDAnalysis as mda
from MDAnalysis.coordinates.XTC import XTCWriter
from MDAnalysis.analysis import align

# Grab all trajectory files you saved
traj_files = sorted(glob("*_backbone.xtc"))

# Load initial universe and get reference positions
u = mda.Universe(topology, traj_files[0])

# Output writer
with XTCWriter("meta_trajectory.xtc", n_atoms=u.atoms.n_atoms) as writer:
    for traj_file in traj_files:
        print(f"Appending: {traj_file}")
        u.load_new(traj_file)
        for ts in u.trajectory:
            writer.write(u.atoms)

# Load your meta-trajectory
u = mda.Universe(topology, "meta_trajectory.xtc")

# Align to first frame using "backbone" or your preferred atom group
aligner = align.AlignTraj(u, u, select="all", in_memory=True)
aligner.run()

# Write aligned frames using a writer
with XTCWriter("meta_trajectory_aligned.xtc", n_atoms=u.atoms.n_atoms) as writer:
    for ts in u.trajectory:
        writer.write(u.atoms)

/opt/anaconda3/envs/hexABC_DB/lib/python3.12/site-packages/MDAnalysis/coordinates/XDR.py:253: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn(


Appending: seq004-1_3_backbone.xtc
Appending: seq004-2_3_backbone.xtc
Appending: seq010-1_15_backbone.xtc
Appending: seq010-2_15_backbone.xtc
Appending: seq011-1_15_backbone.xtc
Appending: seq011-1_2_backbone.xtc
Appending: seq011-2_15_backbone.xtc
Appending: seq011-2_2_backbone.xtc
Appending: seq012-1_2_backbone.xtc
Appending: seq012-2_2_backbone.xtc
Appending: seq015-1_11_backbone.xtc
Appending: seq015-2_11_backbone.xtc
Appending: seq027-1_13_backbone.xtc
Appending: seq027-2_13_backbone.xtc
Appending: seq030-1_5_backbone.xtc
Appending: seq030-2_5_backbone.xtc
Appending: seq036-1_12_backbone.xtc
Appending: seq036-2_12_backbone.xtc
Appending: seq047-1_10_backbone.xtc
Appending: seq047-2_10_backbone.xtc
Appending: seq050-1_10_backbone.xtc
Appending: seq050-2_10_backbone.xtc
Appending: seq055-1_6_backbone.xtc
Appending: seq055-2_6_backbone.xtc
Appending: seq056-1_6_backbone.xtc
Appending: seq056-2_6_backbone.xtc
Appending: seq068-1_10_backbone.xtc
Appending: seq068-2_10_backbone.xtc
Appe

### Visualize meta-trajectory

Representing the **meta-trajectory** with **NGLview**

In [110]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory("meta_trajectory_aligned.xtc", topology), gui=True)
view

NGLWidget(max_frame=82799)

### Clustering the meta-trajectory

**Clustering** the **meta-trajectory** by **RMSd**. Extracting a set of **50 centroids** representing the most populated **conformations** from the chosen **base pair**.

In [88]:
import mdtraj as md
from sklearn.cluster import KMeans
import numpy as np

# Load trajectory
traj = md.load("meta_trajectory_aligned.xtc", top=topology)

# Flatten coordinates for clustering
X = traj.xyz.reshape(traj.n_frames, -1)

# Cluster into k groups
k = 50
kmeans = KMeans(n_clusters=k, random_state=42).fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Find closest frame to each centroid
from scipy.spatial.distance import cdist
closest_frames = np.argmin(cdist(X, centroids), axis=0)

# Extract those frames
centroid_traj = traj.slice(closest_frames)

# Save them into one multi-model PDB file
centroid_traj.save("cluster_centroids.pdb")

### Visualize centroids (conformations)

Representing the **clustering centroids** including the different **conformations** for the specific **base pair** with **NGLview**

In [116]:
view = nglview.show_structure_file("cluster_centroids.pdb", default_representation=False)
view.add_representation(repr_type='ball+stick', selection='all', color='element')
view.center()
view._remote_call('setSize', target='Widget', args=['','400px'])
view


NGLWidget()

### Visualize conformations

Representing the most relevant **conformations** found for the specific **base pair** with **NGLview**.
For this specific case (A**A**T):
- Left: **Watson-Crick** canonical base pairing
- Middle: **Hoogsteen** base pairing
- Right: **Opening**

**Rotating / Translating** in the **left** pannel will trigger the same **rotations / translations** to the **middle** and **right** pannels. 

In [115]:
# Show conformations
view1 = nglview.show_structure_file("cluster_centroids.pdb", default_representation=False)
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.add_representation(repr_type='ball+stick', selection='/1', color='element')
view1.camera='orthographic'
view1.center()
view1
view2 = nglview.show_structure_file("cluster_centroids.pdb", default_representation=False)
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.add_representation(repr_type='ball+stick', selection='/22', color='element')
view2.camera='orthographic'
view2.center()
view2
view3 = nglview.show_structure_file("cluster_centroids.pdb", default_representation=False)
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.add_representation(repr_type='ball+stick', selection='/11', color='element')
view3.camera='orthographic'
view3.center()
view3

def on_change(change):
    view2._set_camera_orientation(change['new'])
    view3._set_camera_orientation(change['new'])
    
view1.observe(on_change, ['_camera_orientation'])

ipywidgets.HBox([view1, view2, view3])